In [ ]:
# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import optuna

# For data preprocess
import numpy as np
import csv
import os

# For plotting
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
# 设置随机种子
myseed = 666
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

# 获取设备
def get_device():
    return 'cuda' if torch.cuda.is_available() else 'cpu'


# 数据处理类
class Dataprocess(Dataset):
    def __init__(self, path, mode='train', modify=True):
        self.mode = mode
        with open(path, 'r') as f:
            data = list(csv.reader(f))
            data = np.array(data[1:])[:, 1:].astype(float)
        if modify == False:
            feats = list(range(0, 93))
        else:
            day1_feats = list(range(40, 58))
            day2_feats = list(range(58, 76))
            day3_feats = list(range(76, 93))
            feats = day1_feats + day2_feats + day3_feats
        if mode == 'test':
            feats = [f + 1 for f in feats]
            data = data[:, feats]
            self.data = torch.FloatTensor(data)
        else:
            target = data[:, -1]
            data = data[:, feats]
            if mode == 'train':
                indices = [i for i in range(len(data)) if i % 10 != 0]
            elif mode == 'dev':
                indices = [i for i in range(len(data)) if i % 10 == 0]
            self.data = torch.FloatTensor(data[indices])
            self.target = torch.FloatTensor(target[indices])
        self.data[:, 40:] = (self.data[:, 40:] - self.data[:, 40:].mean(dim=0, keepdim=True)) / self.data[:, 40:].std(dim=0, keepdim=True)
        self.dim = self.data.shape[1]
        print('Finished reading the {} set of COVID19 Dataset ({} samples found, each dim = {})'.format(mode, len(self.data), self.dim))

    def __getitem__(self, index):
        if self.mode in ['train', 'dev']:
            return self.data[index], self.target[index]
        else:
            return self.data[index]

    def __len__(self):
        return len(self.data)

# 数据加载器
def dataloader(path, mode, batch_size, n_jobs=0, modify=False):
    dataset = Dataprocess(path, mode=mode, modify=modify)
    dataloader = DataLoader(dataset, batch_size, shuffle=(mode == 'train'), drop_last=False, num_workers=n_jobs, pin_memory=True)
    return dataloader

# 神经网络模型
class NeuralNet(nn.Module):
    def __init__(self, input_dim):
        super(NeuralNet, self).__init__()
        tryit = 2048
        self.net = nn.Sequential(
            nn.Linear(input_dim, tryit),
            nn.BatchNorm1d(tryit),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Linear(tryit, 1)
        )
        self.criterion = nn.SmoothL1Loss()

    def forward(self, x):
        return self.net(x).squeeze(1)

    def cal_loss(self, pred, target):
        return self.criterion(pred, target)

# 训练函数
def train(tr_set, dv_set, model, config, device):
    n_epochs = config['n_epochs']
    optimizer = getattr(torch.optim, config['optimizer'])(model.parameters(), **config['optim_hparas'])
    min_mse = 1000.
    loss_record = {'train': [], 'dev': []}
    early_stop_cnt = 0
    epoch = 0
    while epoch < n_epochs:
        model.train()
        for x, y in tr_set:
            optimizer.zero_grad()
            x, y = x.to(device), y.to(device)
            pred = model(x)
            mse_loss = model.cal_loss(pred, y)
            mse_loss.backward()
            optimizer.step()
            loss_record['train'].append(mse_loss.detach().cpu().item())
        dev_mse = dev(dv_set, model, device)
        if dev_mse < min_mse:
            min_mse = dev_mse
            print('Saving model (epoch = {:4d}, loss = {:.4f})'.format(epoch + 1, min_mse))
            torch.save(model.state_dict(), config['save_path'])
            early_stop_cnt = 0
        else:
            early_stop_cnt += 1
        epoch += 1
        loss_record['dev'].append(dev_mse)
        if early_stop_cnt > config['early_stop']:
            break
    print('Finished training after {} epochs'.format(epoch))
    return min_mse, loss_record

# 验证函数
def dev(dv_set, model, device):
    model.eval()
    total_loss = 0
    for x, y in dv_set:
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            pred = model(x)
            mse_loss = model.cal_loss(pred, y)
        total_loss += mse_loss.detach().cpu().item() * len(x)
    total_loss = total_loss / len(dv_set.dataset)
    return total_loss

# 测试函数
def test(tt_set, model, device):
    model.eval()
    preds = []
    for x in tt_set:
        x = x.to(torch.device('cpu'))
        with torch.no_grad():
            pred = model(x)
            preds.append(pred.detach().cpu())
    preds = torch.cat(preds, dim=0).numpy()
    return preds


# 主程序
device = torch.device('cpu')
os.makedirs('models', exist_ok=True)
modify = True
config = {
    'n_epochs': 5000,                
    'batch_size': 256,               
    'optimizer': 'RMSprop',              
    'optim_hparas': {                
        'lr': 0.0005,
        'alpha':0.99, 
        'eps':1e-8,
        'weight_decay': 0.001,         
    },
    'early_stop': 500,               
    'save_path': 'models/model.pth'  
}


train_path = '/Users/johnny/Downloads/24-fall-sdsc-8007-hw-1/HW1.train.csv'  # path to training data
test_path = '/Users/johnny/Downloads/24-fall-sdsc-8007-hw-1/HW1.test.csv'   # path to testing data

train_set = dataloader(train_path, 'train', config['batch_size'], modify=modify)
validation_set = dataloader(train_path, 'dev', config['batch_size'], modify=modify)
test_set = dataloader(test_path, 'test', config['batch_size'], modify=modify)





def objective(trial):
    # 定义超参数搜索空间
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_int('batch_size', 16, 256, log=True)
    n_epochs = trial.suggest_int('n_epochs', 100, 5000)
    
    # 定义模型
    model = NeuralNet(train_set.dataset.dim).to(torch.device('cpu'))
    
    # 定义优化器
    optimizer = torch.optim.RMSprop(model.parameters(), lr=lr)
    
    # 定义数据加载器
    train_loader =  dataloader(train_path, 'train', config['batch_size'], modify=modify)
    val_loader = dataloader(train_path, 'dev', config['batch_size'], modify=modify)

    for epoch in range(n_epochs):
        model.train()
        for x, y in train_loader:
            x, y = x.to('cpu'), y.to('cpu')
            optimizer.zero_grad()
            pred = model(x)
            loss = model.cal_loss(pred, y)
            loss.backward()
            optimizer.step()
    
    # 验证模型
    val_loss = dev(val_loader, model, device)
    
    return val_loss
# 创建Optuna的Study对象
study = optuna.create_study(direction='minimize')
# 运行优化
study.optimize(objective, n_trials=1000)

# 输出最佳超参数
print('Best hyperparameters: ', study.best_params)


[I 2024-09-26 00:02:53,960] A new study created in memory with name: no-name-fb22f273-f2d7-4160-a5a9-45038e0f1f96


Finished reading the train set of COVID19 Dataset (1800 samples found, each dim = 53)
Finished reading the dev set of COVID19 Dataset (200 samples found, each dim = 53)
Finished reading the test set of COVID19 Dataset (650 samples found, each dim = 53)
Finished reading the train set of COVID19 Dataset (1800 samples found, each dim = 53)
Finished reading the dev set of COVID19 Dataset (200 samples found, each dim = 53)


[I 2024-09-26 00:04:13,845] Trial 0 finished with value: 0.7385624647140503 and parameters: {'lr': 2.1174282210628097e-05, 'batch_size': 27, 'n_epochs': 2379}. Best is trial 0 with value: 0.7385624647140503.


Finished reading the train set of COVID19 Dataset (1800 samples found, each dim = 53)
Finished reading the dev set of COVID19 Dataset (200 samples found, each dim = 53)


[I 2024-09-26 00:05:26,376] Trial 1 finished with value: 0.6833949089050293 and parameters: {'lr': 0.04658269276853471, 'batch_size': 22, 'n_epochs': 2148}. Best is trial 1 with value: 0.6833949089050293.


Finished reading the train set of COVID19 Dataset (1800 samples found, each dim = 53)
Finished reading the dev set of COVID19 Dataset (200 samples found, each dim = 53)


[I 2024-09-26 00:07:23,129] Trial 2 finished with value: 0.8816934823989868 and parameters: {'lr': 0.00032080860602083904, 'batch_size': 72, 'n_epochs': 3400}. Best is trial 1 with value: 0.6833949089050293.


Finished reading the train set of COVID19 Dataset (1800 samples found, each dim = 53)
Finished reading the dev set of COVID19 Dataset (200 samples found, each dim = 53)


In [ ]:
import optuna
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import csv
import os

# Set random seed for reproducibility
myseed = 666
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

# Get device
def get_device():
    return 'cuda' if torch.cuda.is_available() else 'cpu'

# Data processing class
class Dataprocess(Dataset):
    def __init__(self, path, mode='train', modify=True):
        self.mode = mode
        with open(path, 'r') as f:
            data = list(csv.reader(f))
            data = np.array(data[1:])[:, 1:].astype(float)
        if modify == False:
            feats = list(range(0, 93))
        else:
            day1_feats = list(range(40, 58))
            day2_feats = list(range(58, 76))
            day3_feats = list(range(76, 93))
            feats = day1_feats + day2_feats + day3_feats
        if mode == 'test':
            feats = [f + 1 for f in feats]
            data = data[:, feats]
            self.data = torch.FloatTensor(data)
        else:
            target = data[:, -1]
            data = data[:, feats]
            if mode == 'train':
                indices = [i for i in range(len(data)) if i % 10 != 0]
            elif mode == 'dev':
                indices = [i for i in range(len(data)) if i % 10 == 0]
            self.data = torch.FloatTensor(data[indices])
            self.target = torch.FloatTensor(target[indices])
        self.data[:, 40:] = (self.data[:, 40:] - self.data[:, 40:].mean(dim=0, keepdim=True)) / self.data[:, 40:].std(dim=0, keepdim=True)
        self.dim = self.data.shape[1]
        print('Finished reading the {} set of COVID19 Dataset ({} samples found, each dim = {})'.format(mode, len(self.data), self.dim))

    def __getitem__(self, index):
        if self.mode in ['train', 'dev']:
            return self.data[index], self.target[index]
        else:
            return self.data[index]

    def __len__(self):
        return len(self.data)

# Data loader
def dataloader(path, mode, batch_size, n_jobs=0, modify=False):
    dataset = Dataprocess(path, mode=mode, modify=modify)
    dataloader = DataLoader(dataset, batch_size, shuffle=(mode == 'train'), drop_last=False, num_workers=n_jobs, pin_memory=True)
    return dataloader

# Neural network model
class NeuralNet(nn.Module):
    def __init__(self, input_dim, n_layers, n_neurons):
        super(NeuralNet, self).__init__()
        layers = []
        for _ in range(n_layers):
            layers.append(nn.Linear(input_dim, n_neurons))
            layers.append(nn.BatchNorm1d(n_neurons))
            layers.append(nn.LeakyReLU(negative_slope=0.01))
            input_dim = n_neurons
        layers.append(nn.Linear(n_neurons, 1))
        self.net = nn.Sequential(*layers)
        self.criterion = nn.SmoothL1Loss()

    def forward(self, x):
        return self.net(x).squeeze(1)

    def cal_loss(self, pred, target):
        return self.criterion(pred, target)

# Training function
def train(tr_set, dv_set, model, config, device):
    n_epochs = config['n_epochs']
    optimizer = getattr(torch.optim, config['optimizer'])(model.parameters(), **config['optim_hparas'])
    min_mse = 1000.
    loss_record = {'train': [], 'dev': []}
    early_stop_cnt = 0
    epoch = 0
    while epoch < n_epochs:
        model.train()
        for x, y in tr_set:
            optimizer.zero_grad()
            x, y = x.to(device), y.to(device)
            pred = model(x)
            mse_loss = model.cal_loss(pred, y)
            mse_loss.backward()
            optimizer.step()
            loss_record['train'].append(mse_loss.detach().cpu().item())
        dev_mse = dev(dv_set, model, device)
        if dev_mse < min_mse:
            min_mse = dev_mse
            print('Saving model (epoch = {:4d}, loss = {:.4f})'.format(epoch + 1, min_mse))
            torch.save(model.state_dict(), config['save_path'])
            early_stop_cnt = 0
        else:
            early_stop_cnt += 1
        epoch += 1
        loss_record['dev'].append(dev_mse)
        if early_stop_cnt > config['early_stop']:
            break
    print('Finished training after {} epochs'.format(epoch))
    return min_mse, loss_record

# Validation function
def dev(dv_set, model, device):
    model.eval()
    total_loss = 0
    for x, y in dv_set:
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            pred = model(x)
            mse_loss = model.cal_loss(pred, y)
        total_loss += mse_loss.detach().cpu().item() * len(x)
    total_loss = total_loss / len(dv_set.dataset)
    return total_loss

# Testing function
def test(tt_set, model, device):
    model.eval()
    preds = []
    for x in tt_set:
        x = x.to(torch.device('cpu'))
        with torch.no_grad():
            pred = model(x)
            preds.append(pred.detach().cpu())
    preds = torch.cat(preds, dim=0).numpy()
    return preds

# Main program
device = torch.device('cpu')
os.makedirs('models', exist_ok=True)
modify = True


train_path = '/Users/johnny/Downloads/24-fall-sdsc-8007-hw-1/HW1.train.csv'  # path to training data
test_path = '/Users/johnny/Downloads/24-fall-sdsc-8007-hw-1/HW1.test.csv'   # path to testing data


def objective(trial):
    # Define hyperparameter search space
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_int('batch_size', 100, 200, log=True)
    n_epochs = trial.suggest_int('n_epochs', 1000, 1500)
    n_layers = trial.suggest_int('n_layers', 1, 1)
    n_neurons = trial.suggest_int('n_neurons', 500, 1500, log=True)
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'AdamW'])

    # Define data loaders
    train_set = dataloader(train_path, 'train', batch_size, modify=modify)
    val_set = dataloader(train_path, 'dev', batch_size, modify=modify)
    
    # Define model
    model = NeuralNet(train_set.dataset.dim, n_layers, n_neurons).to(device)

    # Define optimizer
    optimizer = getattr(torch.optim, optimizer_name)(model.parameters(), lr=lr)

    

    for epoch in range(n_epochs):
        model.train()
        for x, y in train_set:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            pred = model(x)
            loss = model.cal_loss(pred, y)
            loss.backward()
            optimizer.step()

    # Validate model
    val_loss = dev(val_set, model, device)

    return val_loss

# Create Optuna study
study = optuna.create_study(direction='minimize')
# Run optimization
study.optimize(objective, n_trials=100)

# Output best hyperparameters
print('Best hyperparameters: ', study.best_params)


[I 2024-09-26 12:45:54,281] A new study created in memory with name: no-name-9b31783e-6a82-4969-969b-afc944c7d8b7


Finished reading the train set of COVID19 Dataset (1800 samples found, each dim = 53)
Finished reading the dev set of COVID19 Dataset (200 samples found, each dim = 53)


[I 2024-09-26 12:46:21,331] Trial 0 finished with value: 0.619589250087738 and parameters: {'lr': 0.00062222509499192, 'batch_size': 126, 'n_epochs': 1339, 'n_layers': 1, 'n_neurons': 644, 'optimizer': 'Adam'}. Best is trial 0 with value: 0.619589250087738.


Finished reading the train set of COVID19 Dataset (1800 samples found, each dim = 53)
Finished reading the dev set of COVID19 Dataset (200 samples found, each dim = 53)


[I 2024-09-26 12:46:45,826] Trial 1 finished with value: 1.7793535149097444 and parameters: {'lr': 0.06777660655172024, 'batch_size': 166, 'n_epochs': 1242, 'n_layers': 1, 'n_neurons': 675, 'optimizer': 'AdamW'}. Best is trial 0 with value: 0.619589250087738.


Finished reading the train set of COVID19 Dataset (1800 samples found, each dim = 53)
Finished reading the dev set of COVID19 Dataset (200 samples found, each dim = 53)


[I 2024-09-26 12:47:11,608] Trial 2 finished with value: 0.7193797826766968 and parameters: {'lr': 0.015322682048718969, 'batch_size': 104, 'n_epochs': 1066, 'n_layers': 1, 'n_neurons': 671, 'optimizer': 'AdamW'}. Best is trial 0 with value: 0.619589250087738.


Finished reading the train set of COVID19 Dataset (1800 samples found, each dim = 53)
Finished reading the dev set of COVID19 Dataset (200 samples found, each dim = 53)


[I 2024-09-26 12:47:48,784] Trial 3 finished with value: 1.8955415219068528 and parameters: {'lr': 0.04340500500976208, 'batch_size': 115, 'n_epochs': 1260, 'n_layers': 1, 'n_neurons': 1030, 'optimizer': 'AdamW'}. Best is trial 0 with value: 0.619589250087738.


Finished reading the train set of COVID19 Dataset (1800 samples found, each dim = 53)
Finished reading the dev set of COVID19 Dataset (200 samples found, each dim = 53)
